# Spotify.ipynb

This notebook allows the user to search a portion of Spotify's database for a given song and see how Spotify defines various features for the song.

To use, run each cell, then utilize the widgets that the last cell displays to search for songs.

Future tasks:
- Document code
- Abstract functions away, probably in some sort of utils.py file
- Some ways to improve the search feature - more lenient search, search by artist, plot two songs together to compare, etc.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import IPython.display as ipy

In [ ]:
def format_artist(artist):
    return (artist[2:-2].replace("\'", "")).title()

In [ ]:
def select_artist(artists, results):
    artistnames = np.empty_like(artists, dtype=tuple)
    cnt = 0
    for i in artists:
        artistnames[cnt] = (format_artist(i), cnt)
        cnt += 1
    
    interact(plot_song, results=fixed(results), Artist=artistnames);

In [ ]:
def plot_song(results, Artist):
    fig, ax = plt.subplots()

    stats = ('Danceability', 'Energy', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence')
    values = results.loc[:, ['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']]
    values = values.iloc[Artist]
    titlestring = "\"" + (results.iloc[Artist, 1]).title() + "\" by " + ((results.iloc[Artist, 5])[2:-2].replace("\'", "")).title()

    ax.barh(stats, values)
    ax.invert_yaxis()
    ax.set_xlabel('Values')
    ax.set_xlim(left=0.0, right=1.0)
    ax.set_title(titlestring)

    plt.show()

In [ ]:
# Query functions
def submit_query(param):
    ipy.clear_output()
    display(songinput)
    display(button)
    process_query(songinput.value)

def process_query(query):
    # Lower input
    name = query.lower()

    # Lower track names
    lowernames = tracks["name"]
    lowernames = lowernames.str.lower()
    tracks["name"] = lowernames

    # Lower artist names
    lowerartists = tracks["artists"]
    lowerartists = lowerartists.str.lower()
    tracks["artists"] = lowerartists

    # Search query
    songresults = tracks[tracks["name"] == name]
    artistlist = songresults["artists"]

    # Check artist count
    if len(songresults["artists"]) > 1:
        print("Multiple results found. Select an artist:")
        select_artist(artistlist, songresults)
    elif len(songresults["artists"]) < 1:
        print("No results found.")
    else: 
        plot_song(songresults, 0)

In [ ]:
# Query widgets
songinput = widgets.Text(
    description='Song title:',
    disabled=False
)

button = widgets.Button(
    description='Search',
    disabled=False,
    tooltip='Search',
)

In [ ]:
# Open data
tracks = pd.read_csv("data/tracks.csv")

In [ ]:
display(songinput, button)
button.on_click(submit_query)